In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential  # ✅ Corrected
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN  # ✅ Corrected
from tensorflow.keras.layers import Dense, Activation, Dropout  # ✅ Corrected
from tensorflow.keras.layers import Embedding  # ✅ Corrected

from tensorflow.keras.utils import to_categorical  # ✅ Corrected
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer  # ✅ Correct
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


E0000 00:00:1739284946.146078    5597 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:230


In [2]:
import pandas as pd
dtrain = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')
dval = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
dtest = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')

In [3]:
# Initialize TPU with the given name
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local') # Detect TPU
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)

print("✅ TPU Successfully Connected")

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1739284961.739265    5597 service.cc:148] XLA service 0x5835165d8350 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739284961.739312    5597 service.cc:156]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1739284961.739316    5597 service.cc:156]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1739284961.739319    5597 service.cc:156]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1739284961.739322    5597 service.cc:156]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1739284961.739324    5597 service.cc:156]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1739284961.739328    5597 service.cc:156]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1739284961.739332    5597 service.cc:156]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1739284961.739334    5597 service.cc:156]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [4]:
dtrain

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [5]:
dtest

,id,content,lang
0,0,Doctor Who adlı viki başlığına 12. doctor olar...,tr
1,1,"Вполне возможно, но я пока не вижу необходимо...",ru
2,2,"Quindi tu sei uno di quelli conservativi , ...",it
3,3,Malesef gerçekleştirilmedi ancak şöyle bir şey...,tr
4,4,:Resim:Seldabagcan.jpg resminde kaynak sorunu ...,tr
...,...,...,...
63807,63807,"No, non risponderò, come preannunciato. Prefer...",it
63808,63808,"Ciao, I tecnici della Wikimedia Foundation sta...",it
63809,63809,innnazitutto ti ringrazio per i ringraziamenti...,it
63810,63810,Kaç olumlu oy gerekiyor? Şu an 7 oldu. Hayır...,tr


# Data Preprocessing: 

**Consider this example as binary classification with only toxic**

In [6]:
dtrain = dtrain[['id','comment_text','toxic']]
dtrain.head(5)

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0


In [7]:
each_len = []

for k in dtrain['comment_text']:
    each_len.append(len(k.split(' ')))
    
dtrain['comment_len'] = each_len

/tmp/ipykernel_5597/1305875470.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtrain['comment_len'] = each_len


In [8]:
dtrain_sorted = dtrain.groupby('comment_len')[['comment_text']].max().sort_values(by='comment_len', ascending=False).reset_index()

dtrain_sorted

,comment_len,comment_text
0,2834,""" \n = = R e g a r d i n g K a r e l = = ..."
1,2681,""" \n\n ==Hey sexy== \n Quit sucking so much ha..."
2,2321,C O M M I E - F U C K E R C O M M I E - F U C ...
3,2273,"""\n\nAs near as I can tell, wikipedia has no a..."
4,2071,== me editing bad. == \n\n well' i want to be ...
...,...,...
1020,5,일이삼사오육칠팔구하고십이요 에헤헤 으헤 으헤 으허허
1021,4,괴로운 변명을 하는 사람이다-_-;;
1022,3,→ English Wikipedia←
1023,2,ขอบคุณครับคำแนะนำครับ ใส่แหล่งอ้างอิงเรียบร้อย...


In [ ]:
#print(dtrain_sorted.iloc[0]['comment_text'])
#print(dtrain_sorted.iloc[0]['comment_len'])

In [16]:
from sklearn.metrics import roc_auc_score, roc_curve, auc

def roc_auc_cal(pred,y):
    a=roc_auc_score(y, pred)
    return a

In [10]:
#EXAMPLE
'''
# Simulated dataset
y_true = [0, 0, 1, 1]
y_scores = [0.1, 0.4, 0.35, 0.8]

print(roc_auc_cal(y_scores,y_true))  ---- 0.75
'''

'\n# Simulated dataset\ny_true = [0, 0, 1, 1]\ny_scores = [0.1, 0.4, 0.35, 0.8]\n\nprint(roc_auc_cal(y_scores,y_true))  ---- 0.75\n'

In [ ]:
dtrain

In [9]:
#Converting text to lowercase:
import re
def convt_lower(data):
    tmp = []
    for e in data:
        tmp.append(e.lower())
    return tmp

def remove_specialchar(data):
    tmp = []
    for e in data:
        e = e.replace('\n',' ')
        tmp.append(re.sub(r'[^a-zA-Z0-9\s]', '', e))
    return tmp


In [10]:
comment_lc = convt_lower(dtrain['comment_text'])
comment_punc = remove_specialchar(comment_lc)
dtrain['comment_pro']=comment_punc

/tmp/ipykernel_5597/2919373115.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtrain['comment_pro']=comment_punc


**Takeaways here:**

> I need to tokenizer them then i am going to list it into sequence. For doing that, I splitted the data into train and test. But only train data goes to Tokenizer.fit_on_text. If I put train and test data together, tokenization process would be done by considering test data which means the model would see the test data. but it should not see. test data should be unseen.

In [11]:
#first split the data:
y = dtrain['toxic']
x = dtrain['comment_pro']
x_train,x_test,y_train,y_test = train_test_split(x.values,y.values,random_state=42,test_size=0.25, shuffle=True)

maxlen = 2000
#Initialize the tokenizer
token = Tokenizer(num_words=maxlen, oov_token="<OOV>") 
token.fit_on_texts(x_train)
vocab_size=len(token.word_index)

#Apply the tokenizer here:
x_train_seq = token.texts_to_sequences(x_train) 
x_test_seq = token.texts_to_sequences(x_test)   #Truncation just so that max len of data is 2000 words

#Truncation did not take place before. I will to it now!
for e in x_train_seq:
    if len(e)>2000:
        print(len(e))
        print('blinked!')
        break

x_train_pad = pad_sequences(x_train_seq, maxlen=maxlen, padding='post', truncating='post') #post or pre
x_test_pad = pad_sequences(x_test_seq, maxlen=maxlen, padding='post', truncating='post') #post or pre

#now check whether the truncating is done succesfully:
checker = True
for e in x_train_pad:
    if len(e) > 2000:
        print('blinked!')
        checker == False
if checker == True:
    print('truncating is applied sucessfully')
else:
    print('there is a problem in truncating')
    

2142
blinked!
truncating is applied sucessfully


In [12]:
vocab_size=len(token.word_index)
print(vocab_size)

270060


**RNN(Embedding layer needs 3 input):**
- num_vocab
- embedding dim
- inp_len

**Takeaways_1:** (num_vocab)

> Embedding layer needs to know how many words exists in the vocabulary to assign embedding properly. Here created an matrix for the entire vocabulary. Thats why we need make the matrix prepare for how many different inputs will come.
>
**Takeaways_2:** (embedding_dim)

> Embedding layer need to know each token has how many feature vectors. When you assign lets say 100, each token will have 100 different embedding vectors that refers to features.
> Bigger vector of feature, slower training time, better results (also overfiting risk)
> Lesser vector of feature, faster training time, worse results 

In [13]:
loss = 'binary_crossentropy'
optimizer = 'adam'
metrics = ['accuracy']

with strategy.scope():
    model = Sequential()

    model.add(Embedding(vocab_size,100,input_length = maxlen)) #embedding_dim,output_dim,inp_len
    model.add(SimpleRNN(100))
    model.add(Dense(1,activation='sigmoid'))

    model.build(input_shape=(None, 2000))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1739285180.031281    5597 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 2000, 100)      │    27,006,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,026,201 (103.10 MB)

 Trainable params: 27,026,201 (103.10 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(x_train_pad,y_train,epochs = 5, batch_size=128,validation_data = (x_test_pad,y_test))

In [1]:
import tensorflow as tf

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver('local')
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)  # TPU Strategy
    print("✅ TPU Successfully Connected!")
except ValueError:
    strategy = tf.distribute.get_strategy()
    print("❌ TPU Not Found, using default strategy.")


E0000 00:00:1739284895.233048    4345 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:230
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x797bc461e7a0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 757, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
  File "/usr/local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 554, in sigint_handler
    raise KeyboardInterrupt
KeyboardInterrupt: 


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1739284901.931716    4345 service.cc:148] XLA service 0x56fcd0a01f80 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739284901.931765    4345 service.cc:156]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1739284901.931769    4345 service.cc:156]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1739284901.931772    4345 service.cc:156]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1739284901.931775    4345 service.cc:156]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1739284901.931778    4345 service.cc:156]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1739284901.931780    4345 service.cc:156]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1739284901.931783    4345 service.cc:156]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1739284901.931786    4345 service.cc:156]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I